### Sleep state detection

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from neuropy import plotting
import subjects

sessions = subjects.nsd.ratVday1

In [ ]:
from neuropy.analyses import detect_brainstates_epochs

for sub, sess in enumerate(sessions):
    signal = sess.eegfile.get_signal()
    brainstates = detect_brainstates_epochs(
        signal,
        probe=sess.probegroup,
        theta_channel=80,
        delta_channel=82,
        sigma=1,
        ignore_epochs=sess.artifact,
    )
    brainstates.save(sess.filePrefix.with_suffix('.brainstates'))


### View brainstates detection in ephyviewer

In [17]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from neuropy import plotting
from brainstates_editor import editor
from ephyviewer import mkQApp
import subjects

app = mkQApp()
sess = subjects.nsd.ratVday1[0]
sigs = sess.eegfile.get_signal(20)

win = editor(
    sess.brainstates,
    sigs,
    filename=sess.filePrefix.with_suffix(".test"),
    paradigm=sess.paradigm,
)
win.show()
app.exec_()


debug True
QT_MODE PyQt5
refresh duration for traces 8.96453857421875e-05 s
refresh duration for traces 0.00013256072998046875 s
refresh duration for brainstates 0.0003647804260253906 s
refresh duration for traces 0.00019431114196777344 s
refresh duration for brainstates 0.0004432201385498047 s
refresh duration for wvlt 0.00033783912658691406 s
12 6
12 6
12 6
12 6
2500 1250


/home/bapung/Documents/Codes/pythonprogs/DataPaths/spectrogramviewer_custom.py:222: RuntimeWarning: divide by zero encountered in log10
  Sxx = 10.0 * np.log10(Sxx)


2500 1250
refresh duration for traces 0.00011467933654785156 s
refresh duration for brainstates 0.0002143383026123047 s
refresh duration for wvlt 0.000362396240234375 s
refresh duration for spectrogram 0.00016045570373535156 s
2500 1250
2500 1250
refresh duration for traces 0.0001232624053955078 s
refresh duration for brainstates 0.000392913818359375 s
refresh duration for wvlt 0.0001595020294189453 s
refresh duration for spectrogram 0.0003452301025390625 s
refresh duration for paradigm 0.0003247261047363281 s
2500 1250
auto_scale same_for_all
compute_auto_clim
[ True]
2500 1250
2500 1250
refresh duration for traces 0.0001373291015625 s
refresh duration for brainstates 0.0002849102020263672 s
refresh duration for wvlt 0.0005109310150146484 s
refresh duration for2500 spectrogram 0.0006656646728515625 s
 1250
refresh duration for paradigm 0.00034880638122558594 s
refresh duration for traces 0.00013947486877441406 s
refresh duration for brainstates 0.0003838539123535156 s
refresh duration

0

In [ ]:
all_epochs = []
for c in range(3):
    ep_times = np.arange(0, 10., .5) + c*3
    ep_durations = np.ones(ep_times.shape) * .1
    ep_labels = np.array(['Event{} num {}'.format(c, i) for i in range(ep_times.size)], dtype='U')
    all_epochs.append({ 'time':ep_times, 'duration':ep_durations, 'label':ep_labels, 'name':'Event{}'.format(c) })
# source_ep = ephyviewer.InMemoryEpochSource(all_epochs=all_epochs)

In [ ]:
all_epochs[0]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from neuropy import plotting
from ephyviewer import InMemoryAnalogSignalSource
from ephyviewer import mkQApp, MainViewer, TraceViewer
from spectrogramviewer_custom import SpectrogramViewer
import ephyviewer
import subjects

sess = subjects.nsd.ratVday1[0]


# you must first create a main Qt application (for event loop)
app = mkQApp()

# create a fake signals 1 channel at 10kHz
# this emulate a moving frequency
sig = sess.eegfile.get_signal([20, 30], 0, 1000)
sample_rate = sig.sampling_rate
duration = sig.duration
times = sig.time
traces = sig.traces.reshape(-1, 1)
t_start = 0.0
# instantaneous_freqs = np.linspace(500, 3000, times.size)
# instantaneous_phase = np.cumsum(instantaneous_freqs / sample_rate) * 2 * np.pi
# traces[:, 0] += np.sin(instantaneous_phase)


# Create the main window that can contain several viewers
win = MainViewer(debug=True, show_auto_scale=True)

# Create a  signal source for the viewer
source = InMemoryAnalogSignalSource(traces, sample_rate, t_start)

# create a viewer for signal with TraceViewer
view1 = TraceViewer(source=source, name="trace")
view1.params["scale_mode"] = "same_for_all"
view1.params["xsize"] = 3.0
view1.auto_scale()


# create a SpectrogramViewer on the same source
view2 = SpectrogramViewer(source=source, name="spectrogram",freq_lim=(0,40))

view2.params["xsize"] = 30.0
view2.params["colormap"] = "Spectral_r"
view2.params["scalogram", "binsize"] = 2
view2.params["scalogram", "overlapratio"] = 0.5
view2.params["scalogram", "scale"] = "dB"
view2.params["scalogram", "scaling"] = "spectrum"


# add them to mainwindow
win.add_view(view1)
win.add_view(view2)


# show main window and run Qapp
win.show()
app.exec_()


### Sleep fraction during 5 hours following MAZE

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from neuropy import plotting
import subjects

sessions = subjects.sd.brainstates_sess + subjects.nsd.brainstates_sess

In [ ]:
sleep_df = pd.DataFrame()
for sub, sess in enumerate(sessions):
    post= sess.paradigm["post"].flatten()
    period = [post[0],post[0] + 5 * 3600]
    prop = sess.brainstates.get_proportion_by_label(period[0], period[1])
    if "nan" in prop.keys():
        del prop["nan"]

    sleep_df = sleep_df.append(
        pd.DataFrame(
            {
                "prop": list(prop.values()),
                "state": list(prop.keys()),
                "grp":sess.tag,
            }
        ),
        ignore_index=True,
    )

subjects.GroupData().save(sleep_df,'brainstates_proportion')